# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv_file = "../WeatherPy/weather_df.csv"
weather_dataframe = pd.read_csv(weather_csv_file)
weather_dataframe

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,cape town,0,ZA,1650897426,57,-33.9258,18.4232,292.19,9.26
1,1,bahile,71,PH,1650897735,89,9.9995,118.7878,299.31,1.40
2,2,mataura,9,NZ,1650897430,92,-46.1927,168.8643,282.86,2.14
3,3,busselton,100,AU,1650897458,90,-33.6500,115.3333,290.00,8.15
4,4,kapaa,75,US,1650897459,84,22.0752,-159.3190,295.99,6.17
...,...,...,...,...,...,...,...,...,...,...
554,554,ypsonas,0,CY,1650897869,46,34.6884,32.9574,296.69,5.66
555,555,marawi,100,PH,1650897226,94,7.9986,124.2928,294.38,0.80
556,556,chabahar,0,IR,1650897519,68,25.2919,60.6430,301.94,4.46
557,557,kljuc,21,BA,1650897869,52,44.5350,16.7722,290.37,3.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_dataframe[["Lat","Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity = weather_dataframe["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Narrow down the cities to fit weather conditions.
narrow_dataframe = weather_dataframe.loc[(weather_dataframe["Wind Speed"] < 10) & (weather_dataframe["Cloudiness"] < 10) & 
                                   (weather_dataframe["Max Temp"] >= 275) & (weather_dataframe["Max Temp"] <= 300) & (weather_dataframe["Humidity"] <= 30)]

narrow_dataframe.dropna()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
53,53,orebro,0,SE,1650897745,18,59.2741,15.2066,291.62,1.54
109,109,saint george,0,US,1650897259,29,37.1041,-113.5841,280.48,1.54
270,270,aksu,5,CN,1650897451,12,41.1231,80.2644,296.34,9.80
349,349,san juan,0,AR,1650897307,24,-31.5375,-68.5364,297.15,2.84
430,430,leon,0,MX,1650897629,25,21.1167,-101.6667,291.18,1.97
528,528,vredendal,0,ZA,1650897863,27,-31.6683,18.5012,295.41,4.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create a hotel_df
hotel_df = narrow_dataframe.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
53,orebro,SE,59.2741,15.2066,
109,saint george,US,37.1041,-113.5841,
270,aksu,CN,41.1231,80.2644,
349,san juan,AR,-31.5375,-68.5364,
430,leon,MX,21.1167,-101.6667,
528,vredendal,ZA,-31.6683,18.5012,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [12]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)


Retrieving Results for Index 53: orebro.
Closest hotel in orebro is Good Morning Örebro.
------------
Retrieving Results for Index 109: saint george.
Closest hotel in saint george is Hyatt Place St. George / Convention Center.
------------
Retrieving Results for Index 270: aksu.
Closest hotel in aksu is Songhe Hotel.
------------
Retrieving Results for Index 349: san juan.
Closest hotel in san juan is Del Bono Park Hotel Spa & Casino.
------------
Retrieving Results for Index 430: leon.
Closest hotel in leon is Hotel León.
------------
Retrieving Results for Index 528: vredendal.
Closest hotel in vredendal is Tharrakamma Guest House.
------------


In [13]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
53,orebro,SE,59.2741,15.2066,Good Morning Örebro
109,saint george,US,37.1041,-113.5841,Hyatt Place St. George / Convention Center
270,aksu,CN,41.1231,80.2644,Songhe Hotel
349,san juan,AR,-31.5375,-68.5364,Del Bono Park Hotel Spa & Casino
430,leon,MX,21.1167,-101.6667,Hotel León
528,vredendal,ZA,-31.6683,18.5012,Tharrakamma Guest House


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))